In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Polycystic_Ovary_Syndrome"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Polycystic_Ovary_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for Polycystic Ovary Syndrome
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# Polycystic Ovary Syndrome-related keywords
pcos_keywords = ['ovary', 'ovarian', 'polycystic', 'endocrine', 'hormonal', 'reproductive']

# Look for PCOS-related directories
pcos_related_dirs = []
for d in available_dirs:
    if any(keyword in d.lower() for keyword in pcos_keywords):
        pcos_related_dirs.append(d)

print(f"Polycystic Ovary Syndrome-related cohorts: {pcos_related_dirs}")

if not pcos_related_dirs:
    print(f"No suitable cohort found for {trait}.")
    # Mark the task as completed by recording the unavailability
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    # Exit the script early since no suitable cohort was found
    selected_cohort = None
else:
    # Select the most relevant match for PCOS
    # Prioritize directories that mention "ovary" or "ovarian" specifically
    ovarian_specific = [d for d in pcos_related_dirs if 'ovary' in d.lower() or 'ovarian' in d.lower()]
    if ovarian_specific:
        selected_cohort = ovarian_specific[0]
    else:
        # Otherwise select other related cohorts
        selected_cohort = pcos_related_dirs[0]  # Take the first match if multiple exist

if selected_cohort:
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")


Looking for a relevant cohort directory for Polycystic_Ovary_Syndrome...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(CO


Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_OV', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_stage', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'gender', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histol

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify columns that may contain age information
candidate_age_cols = [
    'age_at_initial_pathologic_diagnosis',
    'days_to_birth'
]

# Identify columns that may contain gender information
candidate_gender_cols = [
    'gender'
]

# Import libraries
import pandas as pd
import os

# Get the relevant file paths
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Ovarian_Cancer_(OV)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
if candidate_age_cols:
    age_df = clinical_df[candidate_age_cols]
    print("Age columns preview:")
    print(preview_df(age_df))

# Extract and preview gender columns
if candidate_gender_cols:
    gender_df = clinical_df[candidate_gender_cols]
    print("Gender columns preview:")
    print(preview_df(gender_df))


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [nan, nan, nan, nan, nan], 'days_to_birth': [nan, nan, nan, nan, nan]}
Gender columns preview:
{'gender': [nan, nan, nan, nan, nan]}


### Step 3: Select Demographic Features

In [4]:
# Step: Select Demographic Features

# Analyze the age columns
print("Examining age columns:")
age_columns = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
for col in age_columns:
    if col in clinical_df.columns:
        non_null_count = clinical_df[col].count()
        total_count = len(clinical_df)
        null_percentage = (1 - non_null_count/total_count) * 100
        print(f"Column: {col}")
        print(f"  Non-null values: {non_null_count}/{total_count} ({100-null_percentage:.2f}%)")
        print(f"  First 5 values: {clinical_df[col].head().tolist()}")
        print(f"  Data type: {clinical_df[col].dtype}")
        print()

# Analyze the gender columns
print("Examining gender columns:")
gender_columns = ['gender']
for col in gender_columns:
    if col in clinical_df.columns:
        non_null_count = clinical_df[col].count()
        total_count = len(clinical_df)
        null_percentage = (1 - non_null_count/total_count) * 100
        print(f"Column: {col}")
        print(f"  Non-null values: {non_null_count}/{total_count} ({100-null_percentage:.2f}%)")
        print(f"  First 5 values: {clinical_df[col].head().tolist()}")
        print(f"  Data type: {clinical_df[col].dtype}")
        print()

# Based on preview, select appropriate columns
# If all values are NaN or column doesn't exist, set to None
age_col = None
gender_col = None

# Check if any age column has non-null values
for col in age_columns:
    if col in clinical_df.columns and clinical_df[col].notna().any():
        age_col = col
        break

# Check if any gender column has non-null values
for col in gender_columns:
    if col in clinical_df.columns and clinical_df[col].notna().any():
        gender_col = col
        break

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Examining age columns:
Column: age_at_initial_pathologic_diagnosis
  Non-null values: 607/630 (96.35%)
  First 5 values: [nan, nan, nan, nan, nan]
  Data type: float64

Column: days_to_birth
  Non-null values: 596/630 (94.60%)
  First 5 values: [nan, nan, nan, nan, nan]
  Data type: float64

Examining gender columns:
Column: gender
  Non-null values: 607/630 (96.35%)
  First 5 values: [nan, nan, nan, nan, nan]
  Data type: object

Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
# Use the correct cohort identified in Step 1
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Ovarian_Cancer_(OV)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data if not already loaded
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data from the same cohort directory
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy()  # Keep original orientation for now

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    # First check if we need to transpose based on the data format
    # In TCGA data, typically genes are rows and samples are columns
    if gene_df_for_norm.shape[0] > gene_df_for_norm.shape[1]:
        # More rows than columns, likely genes are rows already
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    else:
        # Need to transpose first
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm.T)
        
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data
        normalized_gene_df = genetic_df
    
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
# In TCGA, samples are typically columns in the gene data and index in the clinical data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Try the alternative orientation
    common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.index))
    print(f"Checking alternative orientation: {len(common_samples)} common samples found.")
    
    if len(common_samples) == 0:
        # Use is_final=False mode which doesn't require df and is_biased
        validate_and_save_cohort_info(
            is_final=False,
            cohort="TCGA",
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True
        )
        print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Ovarian Cancer cohort used for Polycystic Ovary Syndrome gene expression analysis."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Polycystic_Ovary_Syndrome  Age  Gender
sampleID                                               
TCGA-01-0628-11                          0  NaN     NaN
TCGA-01-0629-11                          0  NaN     NaN
TCGA-01-0630-11                          0  NaN     NaN
TCGA-01-0631-11                          0  NaN     NaN
TCGA-01-0633-11                          0  NaN     NaN

Processing gene expression data...


Original gene data shape: (20530, 308)


Attempting to normalize gene symbols...
Gene data shape after normalization: (19848, 308)


Gene data saved to: ../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (630, 3)
Genetic data shape: (19848, 308)
Number of common samples: 308

Linked data shape: (308, 19851)
Linked data preview (first 5 rows, first few columns):
                 Polycystic_Ovary_Syndrome   Age  Gender      A1BG  A1BG-AS1
TCGA-29-1697-01                          1  62.0     0.0 -2.262674 -2.161383
TCGA-25-1630-01                          1  73.0     0.0  0.459726  0.180017
TCGA-29-A5NZ-01                          1  66.0     0.0 -1.364574 -0.924383
TCGA-61-1725-01                          1  40.0     0.0  0.844026 -0.155883
TCGA-61-2008-02                          1  40.0     0.0 -0.700074 -1.640683



Data shape after handling missing values: (308, 19851)

Checking for bias in features:
Quartiles for 'Polycystic_Ovary_Syndrome':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Polycystic_Ovary_Syndrome' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 58.0
  75%: 67.0
Min: 30.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 308 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.




Performing final validation...
The dataset was determined to be unusable for this trait. No data files were saved.
